In [96]:
from tkinter import *
from scipy.signal import convolve2d
import numpy as np
import random
import os

In [75]:
!pip install qiskit --quiet

In [76]:
!pip install qiskit[visualization] --quiet

In [107]:
import qiskit as qk
from qiskit.tools.visualization import circuit_drawer
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators import Operator, Pauli
from scipy.signal import convolve2d
from qiskit.circuit import Parameter
from sympy import *
from qiskit import IBMQ
import math
%matplotlib inline

import matplotlib.pyplot as plt

## Implementation of the Quantum Oracle Connect-n Solver

In [108]:
def place_piece(turn, board, row, col):
    if turn % 2 == 0: board[row-1][col-1] = 'X'
    else: board[row-1][col-1] = 'O'
            
def idx(row, col, n): return n*row+col

def get_min_num_qubits(n):
    search_size = n**2
    num = 0
    while True:
        if search_size <= 2**num: return num
        num += 1
    
def is_power_of_2(n):
    return math.ceil(math.log10(n)/math.log10(2)) == math.floor(math.log10(n)/math.log10(2))

In [109]:
def end_state(board, num_connect, turn):
    board_cpy = [row[:] for row in board]
    for row in range(n):
        for col in range(n):
            if board_cpy[row][col] == 'X': board_cpy[row][col] = 1 if turn % 2 == 0 else 0
            if board_cpy[row][col] == 'O': board_cpy[row][col] = 1 if turn % 2 == 1 else 0
    horizontal_kernel = np.array([[1]*num_connect])
    vertical_kernel = np.transpose(horizontal_kernel)
    diag1_kernel = np.eye(num_connect)
    diag2_kernel = np.fliplr(diag1_kernel)
    detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]
    for kernel in detection_kernels:
        if (convolve2d(board_cpy, kernel, mode="valid") == num_connect).any():
            return True
    return False

def get_int_from_user(message="", lower_bound=1):
    loop_bool = True
    while loop_bool:
        try:
            ret = int(input(message))
            if lower_bound <= ret: loop_bool = False
            else: print("Invalid input: lower bound = {}".format(lower_bound))
        except ValueError: print("Could not read user input")
    return ret

a, b, c, d, e, g, h, i, j,k,l ,m,n,o,p,q,r,s,t,u,v,w,x,y,z= symbols('a b c d e g h i j k m l n o p q r s t u v w x y z')
alphabet= [a,b,c,d,e,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z]
init_printing(use_unicode=True)

In [110]:
def has_token(position, qc_positions, opp_positions):
    for it in qc_positions:
        if (position[0]==it[0]):
            if(position[1]==it[1]):
                return 1
    for it in opp_positions:
        if (position[0]==it[0]):
            if(position[1]==it[1]):
                return -1
    return 0

def change_of_base(position,n):
    qpos=[]
    rows=position[0]
    cols=position[1]
    for it in range(round(math.log(n,2))):
        #Rows
        if(rows>=(n/(2**(it+1)))):
            qpos.append(1)
            rows=rows-(n/(2**(it+1)))
        else:
            qpos.append(0)
        #columns
        if(cols>=(n/(2**(it+1)))):
            qpos.append(1)
            cols=cols-(n/(2**(it+1)))
        else:
            qpos.append(0)
    return qpos

In [111]:
def rev_change_of_base(bitstr, n):
    print(bitstr)
    col=0
    row=0
    counter=0
    pos_counter=2
    for it in bitstr:
        if(pos_counter%2==0):
            if (int(it)==1):
                col+= (2**(counter))
            pos_counter+=1
        else:
            if (int(it)==1):
                row+= (2**(counter))
            counter+=1
            pos_counter+=1
    return (row,col)

def writer(position,n):
    qpos = change_of_base(position,n)
    temp_func = 1
    counter = 0
    for sub_qpos in qpos:
            if sub_qpos==1: temp_func *= alphabet[counter]
            else: temp_func *= (1-alphabet[counter])
            counter+=1
    return temp_func

In [112]:
def Int_cost(position,qc_positions, opp_positions,n):
    inter_cost = 0
    #left edge
    if (position[1]==0):
        #top
        if(position[0]==0):
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]+1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
        #bottom
        elif(position[1]==n-1):
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]+1,qc_positions,opp_positions],n)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]+1],n)
        #everthing else
        else:
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0]+1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
            # 2 BLOCKS
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]-1,position[1]],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]+1,position[1]],n)
    #right edge
    elif(position[1]==n-1):
        #top
        if(position[0]==0):
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]-1],n)
        #bottom
        elif(position[1]==n-1):
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]-1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]-1],n)
        #everthing else
        else:
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]-1],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]-1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]-1],n)   
            # 2 BLOCKS
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]-1,position[1]],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]+1,position[1]],n)
    #everything else
    else:
        #top
        if(position[0]==0):
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]-1],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]+1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
        #bottom
        elif(position[1]==n-1):
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]-1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]-1],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]+1],n)
        #everthing else
        else:
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]-1],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]-1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]-1],n)
            if (has_token([position[0]+1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]+1],n)
            # 2 BLOCKS
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]-1,position[1]],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]+1,position[1]],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0],position[1]-1],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0],position[1]+1],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]+1,position[1]-1],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]-1,position[1]+1],n)
    return inter_cost

In [113]:
def cost_func(qc_positions, opp_positions, n, cost_function=0):
    #qc positions
    for pos in qc_positions:
        temp_func = writer(pos,n)
        #immediate positions
        int_func = Int_cost(pos,qc_positions,opp_positions,n)
        cost_function += -10*temp_func + int_func   
    #player positions
    for pos in opp_positions:
        temp_func = writer(pos,n)
        #immediate positions
        int_func = Int_cost(pos,qc_positions,opp_positions,n)
        cost_function += -20*temp_func + (3/4)*int_func  
    return expand(cost_function)

In [114]:
def get_contribution_to_hamiltonian(to_add, num_qubits):
    sym_map = {a:0, b:1, c:2, d:3, e:4, g:5, h:6, i:7, j:8, k:9, l:10, m:11, n:12, o:13, p:14, q:15, r:16, s:17, t:18, u:19, v:20, w:21, x:22, y:23, z:24}
    inter_str = str(to_add)
    const_idx = 0
    try: const_idx = inter_str.index("*")
    except ValueError: return 0 * Operator(Pauli('I' * num_qubits))
    try: coef = int(inter_str[:const_idx])
    except ValueError: coef = float(inter_str[:const_idx])
    monoms = list(to_add.free_symbols)
    lab = 'I' * num_qubits # monoms = [a, d, f]
    for idx in range(len(monoms)): lab = lab[:sym_map[monoms[idx]]] + 'Z' + lab[sym_map[monoms[idx]] + 1:]
    #len(monoms) = 1 -> local, >1 -> interaction 
    return coef * Operator(Pauli(lab))

In [115]:
def get_hamiltonian_from_cost_func(cost_func_e, num_qubits):
    to_add_list = list(Add.make_args(cost_func_e))
    H_c = Operator(Pauli(label = 'I' * num_qubits))
    for i in range(len(to_add_list)):
        H_c += get_contribution_to_hamiltonian(to_add_list[i], num_qubits)
    return H_c

def create_state_from_outcome(outcome):
    # Turns string of 0s and 1s into quantum state
    zero = np.array([[1],[0]])
    one = np.array([[0],[1]])
    
    state = np.array([[1]])
    for elem in outcome[::-1]: # reverse bit ordering to calculate state
        if elem == '0':
            state = np.kron(state, zero)
        else:
            state = np.kron(state, one)
    return state

def expectation_value(counts, shots, Hc):
    # Computes expectation value with respect to Hc
    hc_matr = Hc.data # change to a numpy array
    exp_val = 0
    for outcome in counts.keys():
        state = create_state_from_outcome(outcome)
        prob = counts[outcome]/shots
        exp_val += prob*(state.conjugate().transpose() @ hc_matr @ state)
    return np.real(exp_val[0,0])

In [116]:
def state_prep(qr,cr):
    circ = qk.QuantumCircuit(qr, cr)
    circ.h(qr)
    circ.barrier()
    return circ

In [122]:
def mixer(beta,qr,cr):
    circ = qk.QuantumCircuit(qr, cr)
    circ.rx(2*beta, qr)
    circ.barrier()
    return circ
def get_quantum_circ(to_add, num_qubits,beta,gamma,qr,cr):
    circ = qk.QuantumCircuit(qr, cr)
    sym_map = {a:0, b:1, c:2, d:3, e:4, g:5, h:6, i:7, j:8, k:9, l:10, m:11, n:12, o:13, p:14, q:15, r:16, s:17, t:18, u:19, v:20, w:21, x:22, y:23, z:24}
    inter_str = str(to_add)
    const_idx = 0
    try: const_idx = inter_str.index("*")
    except ValueError: return 0 * Operator(Pauli('I' * num_qubits))
    try: coef = int(inter_str[:const_idx])
    except ValueError: coef = float(inter_str[:const_idx])
    monoms = list(to_add.free_symbols)
    lab = 'I' * num_qubits # monoms = [a, d, f]
    for idx in range(len(monoms)): lab = lab[:sym_map[monoms[idx]]] + 'Z' + lab[sym_map[monoms[idx]] + 1:]
    #len(monoms) = 1 -> local, >1 -> interaction 
    temp= qk.QuantumCircuit(qr, cr)
    temp.append(Operator(coef*gamma*Pauli(label=lab)),qr)
    print(np.array(Pauli(label=lab)))
    print(lab)
    #temp= qk.transpile(temp, backend=Aer.get_backend('qasm_simulator'), basis_gates=['rz','id','cx','h'])
    temp.draw('mpl', fold=-1, filename="pure_panic")
    return  temp

In [ ]:
def cost(gamma,num_qubits,qr):
    circ = qk.QuantumCircuit(qr)
    
    # Interaction terms
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i == j: continue # sum over i =/= j, skip i == j
            circ.cx(qr[i], qr[j])
            circ.rz(2*gamma/8, qr[j]) # J_{ij} = 1/8 for i != j
            circ.cx(qr[i], qr[j])
    circ.barrier()
    
    # Local terms
    circ.rz(2*2*gamma, qr) # h_i = 2        
    circ.barrier()
    return circ

def choose_best_pos(ham_op, cost_func_e, n):
    num_qubits= get_min_num_qubits(n)
    qr = qk.QuantumRegister(num_qubits)
    cr = qk.ClassicalRegister(num_qubits)
    qaoa_p1 = qk.QuantumCircuit(qr, cr)
    beta = Parameter('b')
    gamma = Parameter('g')
    
    to_add_list = list(Add.make_args(cost_func_e))
    qaoa_p1.compose(state_prep(qr,cr), inplace=True)
    #create cost function   #### didnt have enough time finished
    #for it in range(len(to_add_list)):
        #temp = get_quantum_circ(to_add_list[it], num_qubits,beta,gamma,qr,cr)
        #temp.draw('mpl', fold=-1, filename="pure_panic")
        #qaoa_p1.unitary(temp,qk,label=it)
        
    #mixer function an measures
    qaoa_p1.compose(cost(gamma,n,qr), inplace=True)
    qaoa_p1.compose(mixer(beta,qr,cr), inplace=True)
    qaoa_p1.measure(qr,cr)
    qaoa_p1.draw('mpl', fold=-1, filename="pure_panic")
    beta_val = 3*np.pi/5 # initial choice
    gamma_val = np.pi/2 # initial choice
    qaoa_p1_set = qaoa_p1.bind_parameters({beta: beta_val, gamma: gamma_val})
    shots = 500
    backend = Aer.get_backend('qasm_simulator')
    job = qk.execute(qaoa_p1_set, backend=backend, shots=shots)
    results = job.result()
    counts = results.get_counts(qaoa_p1_set)
    
    beta_list = 2*np.pi*np.linspace(0,1,10)
    gamma_list = np.pi*np.linspace(0,1,10)

    all_data = []
    for beta_val in beta_list:
        gamma_data = []
        for gamma_val in gamma_list:
            qaoa_p1_set = qaoa_p1.bind_parameters({beta: beta_val, gamma: gamma_val})
            job = qk.execute(qaoa_p1_set, backend=backend, shots=shots)
            results = job.result()
            counts = results.get_counts(qaoa_p1_set)
            exp_val = expectation_value(counts, shots, ham_op)
            gamma_data.append(exp_val)
        all_data.append(gamma_data)
        
    opt_val = np.min(all_data) # we want the max <H_C>
    for i in range(len(all_data)):
        for j in range(len(all_data[i])):
            if all_data[i][j] == opt_val:
                opt_i = i
                opt_j = j
                break
    opt_beta = beta_list[opt_i]
    opt_gamma = gamma_list[opt_j]
    
    qaoa_p1_set = qaoa_p1.bind_parameters({beta: beta_val, gamma: gamma_val})
    job = qk.execute(qaoa_p1_set, backend=backend, shots=shots)
    results = job.result()
    counts = results.get_counts(qaoa_p1_set)
    best_bitstr = max(counts, key = counts.get)
    plot_histogram(counts)
    row, col =rev_change_of_base(best_bitstr, n)
    row =int(row+1)
    col =int(col+1)
    return row,col,counts

## Non-graphical mode of Game Design
Run this version instead of the interactive one below.

In [ ]:
import time

def game_loop():
    global n, board, human_pos, machine_pos
    human_pos = []
    machine_pos = []
    n = get_int_from_user(message="Board length/width (N x N sized board): ", lower_bound=2)
    while not is_power_of_2(n):
        print("N must be a power of 2! (eg. N = 2, 4, 8, etc.)")
        n = get_int_from_user(message="Board length/width (N x N sized board): ", lower_bound=2)
    num_qubits = get_min_num_qubits(n)
    num_connect = get_int_from_user(message="Number of blocks to connect to win (X in \"connect-X\"): ", lower_bound=3)
    board = [[0]*n for _ in range(n)]
    turn = 0
    
    while not end_state(board, num_connect, turn):
        print("Turn: {}".format("Human Player" if turn % 2 == 0 else "Machine Player"))
        print(np.array(board))
        if turn % 2 == 0: # Human turn
            row = get_int_from_user(message="Row Number [1 to {}, inclusive]: ".format(n))
            col = get_int_from_user(message="Column Number [1 to {}, inclusive]: ".format(n))
            human_pos.append([row, col])
        else: # Machine turn
            # get row, col from quantum algorithm
            cost = cost_func(machine_pos, human_pos, n)
            ham = get_hamiltonian_from_cost_func(cost, num_qubits)
            row, col, counts = choose_best_pos(ham, cost, n)
            print(row,col)
            plot_histogram(counts, filename="./PANIC.png")
            machine_pos.append([row, col])
            
        place_piece(turn, board, row, col)
        turn += 1 
        
game_loop()

## Game Implementation with Interactive UI

In [26]:
## set the grid map size and the number of connected tokens for winning condition
number_grids = 8
number_connects = 5
if number_connects > number_grids:
    print("The number of connected tokes has to be smaller or equal to the map size.")

In [27]:
size_of_board = 600
symbol_size = (size_of_board / number_grids - size_of_board / (2*number_grids + 2)) / 2
symbol_thickness = 15
symbol_X_color = '#EE4035'
symbol_O_color = '#0492CF'
Green_color = '#7BC043'

In [ ]:
class Connect_Five():
    # ------------------------------------------------------------------
    # Initialization Functions:
    # ------------------------------------------------------------------
    def __init__(self):
        self.window = Tk()
        self.window.title('Connect Five')
        self.canvas = Canvas(self.window, width=size_of_board*2, height=size_of_board)
        self.canvas.pack()
        # Input from user in form of clicks
        self.window.bind('<Button-1>', self.click)

        self.display_rules()
        self.player_X_turns = True
        self.board_status = np.zeros(shape=(number_grids, number_grids))

        self.player_X_starts = True
        self.reset_board = True
        self.pre_initialize = True
        self.gameover = False
        self.tie = False
        self.X_wins = False
        self.O_wins = False

        self.X_score = 0
        self.O_score = 0
        self.tie_score = 0
        self.human_pos = []
        self.machine_pos = []
        self.photo = PhotoImage(file="QM_circuit_imgs/nyan.png")

    def mainloop(self):
        self.window.mainloop()

    def initialize_board(self):
        self.canvas.delete("all")
        for i in range(number_grids):
            self.canvas.create_line((i + 1) * size_of_board / number_grids, 0, (i + 1) * size_of_board / number_grids, size_of_board)

        for i in range(number_grids - 1):
            self.canvas.create_line(0, (i + 1) * size_of_board / number_grids, size_of_board, (i + 1) * size_of_board / number_grids)
        
        color = 'gray'
        circuit_text_1 = "The quantum logic circuit used by"
        circuit_text_2 = "the computer to solve the problem:"

        # Add the image in the canvas
        self.canvas.create_image(size_of_board * 1.5, 14 * size_of_board / 30, image=self.photo, anchor="center")
        
        self.canvas.create_text(size_of_board * 1.5, 4 * size_of_board / 30, font="cmr 30 bold", fill=color, text=circuit_text_1)
        self.canvas.create_text(size_of_board * 1.5, 6 * size_of_board / 30, font="cmr 30 bold", fill=color, text=circuit_text_2)
#         self.canvas.create_image(size_of_board * 1.5, 8 * size_of_board / 30, image=img)

    def play_again(self):
        self.initialize_board()
#         self.player_X_starts = not self.player_X_starts
        self.player_X_turns = self.player_X_starts
        self.board_status = np.zeros(shape=(number_grids, number_grids))
    # ------------------------------------------------------------------
    # Drawing Functions:
    # The modules required to draw required game based object on canvas
    # ------------------------------------------------------------------

    def draw_O(self, logical_position):
        logical_position = np.array(logical_position)
        # logical_position = grid value on the board
        # grid_position = actual pixel values of the center of the grid
        grid_position = self.convert_logical_to_grid_position(logical_position)
        self.canvas.create_oval(grid_position[0] - symbol_size, grid_position[1] - symbol_size,
                                grid_position[0] + symbol_size, grid_position[1] + symbol_size, width=symbol_thickness,
                                outline=symbol_O_color)

    def draw_X(self, logical_position):
        grid_position = self.convert_logical_to_grid_position(logical_position)
        self.canvas.create_line(grid_position[0] - symbol_size, grid_position[1] - symbol_size,
                                grid_position[0] + symbol_size, grid_position[1] + symbol_size, width=symbol_thickness,
                                fill=symbol_X_color)
        self.canvas.create_line(grid_position[0] - symbol_size, grid_position[1] + symbol_size,
                                grid_position[0] + symbol_size, grid_position[1] - symbol_size, width=symbol_thickness,
                                fill=symbol_X_color)

    def display_rules(self):
        
        self.canvas.create_line((number_grids + 1) * size_of_board / number_grids, 0, (number_grids + 1) * size_of_board / number_grids, size_of_board)
        
        color = 'gray'
        rules_text_1 = "Rules of this game:"
        rules_text_2 = "1. Hover above a tile and click to make your first move."
        rules_text_3 = "Your moves will be marked in X."
        rules_text_4 = "2. Click anywhere on screen to let the computer make its move."
        rules_text_5 = "The Computer's moves will be marked in O."
        rules_text_6 = "3. Whoever connects the desired number of tiles first wins."
        score_text = 'Click to play'
        
        self.canvas.create_text(size_of_board / 2, 6 * size_of_board / 30, font="cmr 60 bold", fill=color, text=rules_text_1)
        self.canvas.create_text(size_of_board / 2, 12 * size_of_board / 30, font="cmr 20 bold", fill=Green_color,
                                text=rules_text_2)
        self.canvas.create_text(size_of_board / 2, 14 * size_of_board / 30, font="cmr 20 bold", fill=Green_color,
                                text=rules_text_3)
        self.canvas.create_text(size_of_board / 2, 16 * size_of_board / 30, font="cmr 20 bold", fill=Green_color,
                                text=rules_text_4)
        self.canvas.create_text(size_of_board / 2, 18 * size_of_board / 30, font="cmr 20 bold", fill=Green_color,
                                text=rules_text_5)
        self.canvas.create_text(size_of_board / 2, 20 * size_of_board / 30, font="cmr 20 bold", fill=Green_color,
                                text=rules_text_6)
        
        self.canvas.create_text(size_of_board / 2, 24 * size_of_board / 30, font="cmr 40 bold", fill=color,
                                text=score_text)
        
    def display_gameover(self):

        if self.X_wins:
            self.X_score += 1
            text = 'Winner: Player 1 (X)'
            color = symbol_X_color
        elif self.O_wins:
            self.O_score += 1
            text = 'Winner: Player 2 (O)'
            color = symbol_O_color
        else:
            self.tie_score += 1
            text = 'Its a tie'
            color = 'gray'

        self.canvas.delete("all")
        self.canvas.create_line((number_grids + 1) * size_of_board / number_grids, 0, (number_grids + 1) * size_of_board / number_grids, size_of_board)
        self.canvas.create_text(size_of_board / 2, size_of_board / 3, font="cmr 60 bold", fill=color, text=text)

        score_text = 'Scores \n'
        self.canvas.create_text(size_of_board / 2, 5 * size_of_board / 8, font="cmr 40 bold", fill=Green_color,
                                text=score_text)

        score_text = 'Player 1 (X) : ' + str(self.X_score) + '\n'
        score_text += 'Player 2 (O): ' + str(self.O_score) + '\n'
        score_text += 'Tie                    : ' + str(self.tie_score)
        self.canvas.create_text(size_of_board / 2, 3 * size_of_board / 4, font="cmr 30 bold", fill=Green_color,
                                text=score_text)
        self.reset_board = True

        score_text = 'Click to play again \n'
        self.canvas.create_text(size_of_board / 2, 15 * size_of_board / 16, font="cmr 20 bold", fill="gray",
                                text=score_text)

    # ------------------------------------------------------------------
    # Logical Functions:
    # The modules required to carry out game logic
    # ------------------------------------------------------------------
    
    # placeholder for the real quantum oracle function for determining where to place the token
    def qc_oracle_place_holder(self):
        pos_x = random.randint(0, number_grids-1)
        pos_y = random.randint(0, number_grids-1)
        logical_position = [pos_x, pos_y]
        occupied = True
        if self.board_status[logical_position[0]][logical_position[1]] == 0:
            occupied = False
        while occupied == True:
            pos_x = random.randint(0, number_grids-1)
            pos_y = random.randint(0, number_grids-1)
            logical_position = [pos_x, pos_y]
            if self.board_status[logical_position[0]][logical_position[1]] == 0:
                occupied = False
        return logical_position
    
    def real_qc_oracle(self):
        n = number_grids
        num_qubits = get_min_num_qubits(n)
        cost = cost_func(self.machine_pos, self.human_pos, n)
        ham = get_hamiltonian_from_cost_func(cost, num_qubits)
        row, col, counts = choose_best_pos(ham, cost, n)
        self.machine_pos.append([row, col])
        return [row, col]

    def convert_logical_to_grid_position(self, logical_position):
        logical_position = np.array(logical_position, dtype=int)
        return (size_of_board / number_grids) * logical_position + size_of_board / (2*number_grids)

    def convert_grid_to_logical_position(self, grid_position):
        grid_position = np.array(grid_position)
        return np.array(grid_position // (size_of_board / number_grids), dtype=int)
    
    def is_grid_inbound(self, logical_position):
        if (logical_position[0] in range(number_grids)) and (logical_position[1] in range(number_grids)):
            return True
        else:
            return False

    def is_grid_occupied(self, logical_position):
        if self.board_status[logical_position[0]][logical_position[1]] == 0:
            return False
        else:
            return True      

    def is_winner(self, player):
        # create convolution kernels correspondent to connecting patterns of m tiles horizontally, vertically and diagonally
        horizontal_kernel = np.ones((1,number_connects))
        vertical_kernel = np.transpose(horizontal_kernel)
        diag1_kernel = np.eye(number_connects, dtype=np.uint8)
        diag2_kernel = np.fliplr(diag1_kernel)
        detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]

        player = -1 if player == 'X' else 1
        
        # represent the board in a 2d array such that all empty tiles or tiles occupied by opponent are set to 0
        board_representation = np.where(self.board_status == player, 1, 0)
        
        # convolution kernels matched either horizontal, vertical or diagonal lines of tiles needed to win
        for kernel in detection_kernels:
            if (convolve2d(board_representation, kernel, mode="valid") == number_connects).any():
                return True
        return False

    def is_tie(self):

        r, c = np.where(self.board_status == 0)
        tie = False
        if len(r) == 0:
            tie = True

        return tie

    def is_gameover(self):
        # Either someone wins or all grid occupied
        self.X_wins = self.is_winner('X')
        if not self.X_wins:
            self.O_wins = self.is_winner('O')

        if not self.O_wins:
            self.tie = self.is_tie()

        gameover = self.X_wins or self.O_wins or self.tie

        if self.X_wins:
            print('X wins')
        if self.O_wins:
            print('O wins')
        if self.tie:
            print('Its a tie')

        return gameover

    def click(self, event):
        grid_position = [event.x, event.y]
        logical_position = self.convert_grid_to_logical_position(grid_position)

        if not self.reset_board and not self.pre_initialize:
            if self.player_X_turns:
                if self.is_grid_inbound(logical_position) and not self.is_grid_occupied(logical_position):
                    self.draw_X(logical_position)
                    self.board_status[logical_position[0]][logical_position[1]] = -1
                    self.player_X_turns = not self.player_X_turns
                    self.human_pos.append(logical_position)
            else:
                if self.is_grid_inbound(logical_position) and not self.is_grid_occupied(logical_position):
                    logical_position = self.real_qc_oracle()
                    self.draw_O(logical_position)
                    self.board_status[logical_position[0]][logical_position[1]] = 1
                    self.player_X_turns = not self.player_X_turns

            # Check if game is concluded
            if self.is_gameover():
                self.display_gameover()
                # print('Done')
        elif self.pre_initialize and self.reset_board:  # Display Rules pre-game
            self.canvas.delete("all")
            self.display_rules()
            self.pre_initialize = False
            self.reset_board = False
            self.initialize_board()
        else: # Play Again
            self.canvas.delete("all")
            self.play_again()
            self.reset_board = False


game_instance = Connect_Five()
game_instance.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/audreina17/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/dk/vprrswhx0rx51wzzt5p923_w0000gn/T/ipykernel_56059/1352956649.py", line 255, in click
    logical_position = self.real_qc_oracle()
  File "/var/folders/dk/vprrswhx0rx51wzzt5p923_w0000gn/T/ipykernel_56059/1352956649.py", line 170, in real_qc_oracle
    row, col, counts = choose_best_pos(ham, cost, n)
  File "/var/folders/dk/vprrswhx0rx51wzzt5p923_w0000gn/T/ipykernel_56059/1684982509.py", line 35, in choose_best_pos
    qaoa_p1.compose(cost(gamma,n,qr), inplace=True)
  File "/var/folders/dk/vprrswhx0rx51wzzt5p923_w0000gn/T/ipykernel_56059/1684982509.py", line 8, in cost
    circ.cx(qr[i], qr[j])
  File "/Users/audreina17/opt/anaconda3/lib/python3.9/site-packages/qiskit/circuit/register.py", line 173, in __getitem__
    return self._bits[key]
IndexError: list index out of range
